In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
        print(dirname)

# I’m Something of a Painter Myself  
### Mini-Project: Generative Adversarial Networks for Artistic Style Generation

This mini-project is focused on building and training deep generative learning models. The goal is to gain hands-on experience with Generative Adversarial Networks (GANs) by applying them to an image generation task inspired by artistic style transfer. The project uses a Kaggle “Getting Started” competition as a practical framework for model development, evaluation, and comparison.

---

## Problem Description

The objective of this project is to build a generative model capable of producing images that resemble the artistic style of Claude Monet. Specifically, the task is to train a Generative Adversarial Network (GAN) to generate novel images that capture characteristic features of Monet’s paintings, such as color palettes, textures, and brush-stroke patterns.

GANs consist of two competing neural networks: a generator, which attempts to create realistic images, and a discriminator, which learns to distinguish between real images and generated ones. Through this adversarial training process, the generator progressively improves its ability to produce high-quality, realistic images.  

This problem is relevant because it demonstrates how modern deep learning models can learn complex visual distributions and creative styles, highlighting the intersection between computer vision, unsupervised learning, and computational creativity.


## Data Description

The dataset used in this project consists of image data provided by the Kaggle *“I’m Something of a Painter Myself”* competition. The data is designed for training generative models to learn and reproduce an artistic style.

- **Data type**:  
  - Monet paintings (target domain), representing the artistic style to be learned.  
  - Photographs (source domain), which can be used as input images when applying style transfer approaches such as CycleGAN.

- **Image format**:  
  - Images are RGB with shape **256 × 256 × 3** after preprocessing/resizing (as required for model training and submission).

- **Data size (as observed in this notebook environment)**:  
  - **Monet JPG** folder: **300** images  
  - **Photo JPG** folder: **7,038** images

- **Data structure**:  
  - Images are available both as **JPG files** (`monet_jpg/`, `photo_jpg/`) and as **TFRecords** (`monet_tfrec/`, `photo_tfrec/`) for more efficient TensorFlow input pipelines.  
  - No labels are provided, since this is an **unsupervised** image generation / style learning task.

Overall, the dataset is suitable for training GAN-based models to learn the visual distribution of Monet-style paintings and generate new images consistent with that style.


In [ ]:
# =========================
# Data Loading + EDA (CPU-only)
# =========================

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"   # 0=all, 1=filter INFO, 2=filter WARNING, 3=filter ERROR

import glob, random
from pathlib import Path

import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# -------------------------
# Paths (given)
# -------------------------
ROOT = Path("/kaggle/input/gan-getting-started")
MONET_JPG_DIR   = ROOT / "monet_jpg"
PHOTO_JPG_DIR   = ROOT / "photo_jpg"
MONET_TFREC_DIR = ROOT / "monet_tfrec"
PHOTO_TFREC_DIR = ROOT / "photo_tfrec"

print("Root:", ROOT)
print("Monet JPG:", MONET_JPG_DIR)
print("Photo JPG:", PHOTO_JPG_DIR)
print("Monet TFRecords:", MONET_TFREC_DIR)
print("Photo TFRecords:", PHOTO_TFREC_DIR)

# -------------------------
# Helpers
# -------------------------
def list_files(dir_path: Path, exts=(".jpg", ".jpeg", ".png")):
    files = []
    for ext in exts:
        files.extend(sorted(glob.glob(str(dir_path / f"*{ext}"))))
    return files

def show_grid_from_paths(paths, rows=2, cols=3, title=None):
    """Show a small fixed grid (default 3x2) to avoid excessive visualization."""
    paths = list(paths)
    n = rows * cols
    if len(paths) == 0:
        print(f"[WARN] No images to display for: {title}")
        return

    sample = random.sample(paths, k=min(n, len(paths)))
    plt.figure(figsize=(cols * 4, rows * 4))
    if title:
        plt.suptitle(title, fontsize=14)

    for i, p in enumerate(sample, start=1):
        img = Image.open(p).convert("RGB")
        w, h = img.size
        plt.subplot(rows, cols, i)
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"{w}x{h}", fontsize=10)

    plt.tight_layout()
    plt.show()

def check_all_sizes(paths, expected_w=256, expected_h=256, label=""):
    """
    Verify that ALL images match expected size.
    Reports width check and height check separately + up to 10 mismatches.
    """
    paths = list(paths)
    if len(paths) == 0:
        print(f"[WARN] No images found for {label}")
        return

    width_ok = True
    height_ok = True
    mismatches = []

    for p in paths:
        with Image.open(p) as img:
            w, h = img.size
        if w != expected_w:
            width_ok = False
        if h != expected_h:
            height_ok = False
        if (w != expected_w) or (h != expected_h):
            mismatches.append((p, w, h))

    print(f"\n[{label}] Total images: {len(paths)}")
    print(f"[{label}] All widths == {expected_w}: {width_ok}")
    print(f"[{label}] All heights == {expected_h}: {height_ok}")

    if mismatches:
        print(f"[{label}] Mismatches found: {len(mismatches)} (showing up to 10)")
        for p, w, h in mismatches[:10]:
            print(f"  - {p} -> {w}x{h}")
    else:
        print(f"[{label}] No size mismatches found.")

# -------------------------
# Load JPG file lists
# -------------------------
monet_jpg_paths = list_files(MONET_JPG_DIR, exts=(".jpg", ".jpeg"))
photo_jpg_paths = list_files(PHOTO_JPG_DIR, exts=(".jpg", ".jpeg"))

print(f"Monet JPG count: {len(monet_jpg_paths)}")
print(f"Photo JPG count: {len(photo_jpg_paths)}")

# -------------------------
# TFRecords discovery (informational)
# -------------------------
monet_tfrec_files = sorted(glob.glob(str(MONET_TFREC_DIR / "*.tfrec"))) + sorted(glob.glob(str(MONET_TFREC_DIR / "*.tfrecord*")))
photo_tfrec_files = sorted(glob.glob(str(PHOTO_TFREC_DIR / "*.tfrec"))) + sorted(glob.glob(str(PHOTO_TFREC_DIR / "*.tfrecord*")))

print(f"Monet TFRecord files: {len(monet_tfrec_files)}")
print(f"Photo TFRecord files: {len(photo_tfrec_files)}")

# -------------------------
# EDA: show few sample images (3x2 each)
# -------------------------
show_grid_from_paths(monet_jpg_paths, rows=2, cols=3, title="Monet JPG Samples (3x2)")
show_grid_from_paths(photo_jpg_paths, rows=2, cols=3, title="Photo JPG Samples (3x2)")

# -------------------------
# EDA: size checks only 
# -------------------------
check_all_sizes(monet_jpg_paths, expected_w=256, expected_h=256, label="Monet JPG")
check_all_sizes(photo_jpg_paths, expected_w=256, expected_h=256, label="Photo JPG")

# -------------------------
# Preprocessing functions 
# -------------------------
IMG_SIZE = 256

def decode_and_resize_jpg(path, img_size=IMG_SIZE):
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img_bytes, channels=3)
    img = tf.image.resize(img, [img_size, img_size], method="bilinear")
    img = tf.cast(img, tf.float32)
    return img

def normalize_01(img):
    return img / 255.0

def normalize_m11(img):
    return (img / 127.5) - 1.0

# -------------------------
# Quick check: dtype/range after preprocessing 
# -------------------------
def inspect_preprocessing(paths, n=4, label=""):
    if len(paths) == 0:
        return
    sample = random.sample(list(paths), k=min(n, len(paths)))
    imgs = [decode_and_resize_jpg(p) for p in sample]
    batch = tf.stack(imgs, axis=0)  # [B, H, W, C]

    batch_01 = normalize_01(batch)
    batch_m11 = normalize_m11(batch)

    print(f"\n[{label}] Raw batch:", batch.shape, batch.dtype,
          "min/max:", float(tf.reduce_min(batch)), float(tf.reduce_max(batch)))
    print(f"[{label}] 0..1 batch:", batch_01.shape, batch_01.dtype,
          "min/max:", float(tf.reduce_min(batch_01)), float(tf.reduce_max(batch_01)))
    print(f"[{label}] -1..1 batch:", batch_m11.shape, batch_m11.dtype,
          "min/max:", float(tf.reduce_min(batch_m11)), float(tf.reduce_max(batch_m11)))

inspect_preprocessing(monet_jpg_paths, n=4, label="Monet")
inspect_preprocessing(photo_jpg_paths, n=4, label="Photo")

# -------------------------
# Build tf.data pipelines (JPG) for training (CPU)
# -------------------------
AUTOTUNE = tf.data.AUTOTUNE

def make_jpg_dataset(paths, batch_size=16, shuffle=True, repeat=True, norm="m11"):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(paths), 2048), reshuffle_each_iteration=True)
    ds = ds.map(lambda p: decode_and_resize_jpg(p, IMG_SIZE), num_parallel_calls=AUTOTUNE)
    if norm == "m11":
        ds = ds.map(normalize_m11, num_parallel_calls=AUTOTUNE)
    elif norm == "01":
        ds = ds.map(normalize_01, num_parallel_calls=AUTOTUNE)
    if repeat:
        ds = ds.repeat()
    ds = ds.batch(batch_size, drop_remainder=True).prefetch(AUTOTUNE)
    return ds

# Example datasets (for later training)
monet_ds = make_jpg_dataset(monet_jpg_paths, batch_size=16, shuffle=True, repeat=True, norm="m11")
photo_ds = make_jpg_dataset(photo_jpg_paths, batch_size=16, shuffle=True, repeat=True, norm="m11")

# Peek one batch
m_batch = next(iter(monet_ds.take(1)))
p_batch = next(iter(photo_ds.take(1)))
print("\nMonet batch:", m_batch.shape, m_batch.dtype, "min/max:", float(tf.reduce_min(m_batch)), float(tf.reduce_max(m_batch)))
print("Photo batch:", p_batch.shape, p_batch.dtype, "min/max:", float(tf.reduce_min(p_batch)), float(tf.reduce_max(p_batch)))


## Model Choice and Motivation

For this project, a **Generative Adversarial Network (GAN)** architecture is required to generate images in the style of Claude Monet. Among the available GAN-based approaches, **CycleGAN** is a natural and well-established choice for this task.

CycleGAN is designed for **unpaired image-to-image translation**, meaning it can learn a mapping between two visual domains without requiring one-to-one correspondence between images. In this case, the two domains are:
- photographs (source domain)
- Monet-style paintings (target domain)

This property is particularly important because the dataset does not provide paired photo–painting examples. CycleGAN addresses this by using two generators and two discriminators, along with a **cycle-consistency constraint**, which encourages the translated images to preserve the underlying content of the original images while adopting the target style.

The use of CycleGAN allows the model to:
- learn stylistic features such as color distributions and texture patterns from Monet paintings,
- preserve structural content from photographs,
- operate in a fully unsupervised setting, consistent with the dataset characteristics.

Given these properties, CycleGAN provides a robust and conceptually appropriate baseline for learning Monet-style image generation in this mini-project.


In [ ]:
# =========================
# CycleGAN - Model Architecture (tf.keras) 
# Generators: ResNet-style with Conv(stride=2) downsampling + Residual blocks + UpSampling
# Discriminators: PatchGAN producing logits (no sigmoid) for BCEWithLogits
# =========================

import tensorflow as tf

# -------------------------
# Building Blocks (merged conv block)
# -------------------------
def conv_norm_act(
    x,
    filters,
    kernel_size=3,
    strides=1,
    padding="same",
    use_norm=True,
    act="relu",          # "relu" or "lrelu"
    lrelu_alpha=0.2,
    name=None,
):
    x = tf.keras.layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        use_bias=not use_norm,
        kernel_initializer="he_normal",
        name=None if name is None else f"{name}_conv",
    )(x)

    if use_norm:
        x = tf.keras.layers.BatchNormalization(
            name=None if name is None else f"{name}_bn"
        )(x)

    if act == "relu":
        x = tf.keras.layers.ReLU(name=None if name is None else f"{name}_relu")(x)
    elif act == "lrelu":
        x = tf.keras.layers.LeakyReLU(lrelu_alpha, name=None if name is None else f"{name}_lrelu")(x)
    else:
        raise ValueError("act must be 'relu' or 'lrelu'")

    return x

def residual_block(x, filters, name=None):
    """
    Minimal ResNet block for generator:
    x -> Conv -> BN -> ReLU -> Conv -> BN -> Add(skip)
    """
    skip = x

    y = tf.keras.layers.Conv2D(
        filters, 3, strides=1, padding="same", use_bias=False,
        kernel_initializer="he_normal",
        name=None if name is None else f"{name}_c1",
    )(x)
    y = tf.keras.layers.BatchNormalization(name=None if name is None else f"{name}_bn1")(y)
    y = tf.keras.layers.ReLU(name=None if name is None else f"{name}_relu")(y)

    y = tf.keras.layers.Conv2D(
        filters, 3, strides=1, padding="same", use_bias=False,
        kernel_initializer="he_normal",
        name=None if name is None else f"{name}_c2",
    )(y)
    y = tf.keras.layers.BatchNormalization(name=None if name is None else f"{name}_bn2")(y)

    out = tf.keras.layers.Add(name=None if name is None else f"{name}_add")([skip, y])
    return out

# -------------------------
# Generator (ResNet-style)
# -------------------------
def build_generator(name="generator", img_size=256, base_filters=64, n_res_blocks=6):
    """
    Input:  (img_size, img_size, 3) normalized to [-1, 1]
    Output: (img_size, img_size, 3) with tanh in [-1, 1]
    """
    inp = tf.keras.layers.Input(shape=(img_size, img_size, 3), name=f"{name}_input")

    # Initial conv
    x = conv_norm_act(inp, base_filters, kernel_size=7, strides=1, padding="same",
                      use_norm=True, act="relu", name=f"{name}_init")

    # Downsampling: 256 -> 128 -> 64
    x = conv_norm_act(x, base_filters * 2, kernel_size=3, strides=2, padding="same",
                      use_norm=True, act="relu", name=f"{name}_down1")
    x = conv_norm_act(x, base_filters * 4, kernel_size=3, strides=2, padding="same",
                      use_norm=True, act="relu", name=f"{name}_down2")

    # Residual blocks at 64x64
    for i in range(n_res_blocks):
        x = residual_block(x, base_filters * 4, name=f"{name}_res{i+1}")

    # Upsampling: 64 -> 128 -> 256 (nearest + conv)
    x = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation="nearest", name=f"{name}_up1")(x)
    x = conv_norm_act(x, base_filters * 2, kernel_size=3, strides=1, padding="same",
                      use_norm=True, act="relu", name=f"{name}_up1_conv")

    x = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation="nearest", name=f"{name}_up2")(x)
    x = conv_norm_act(x, base_filters, kernel_size=3, strides=1, padding="same",
                      use_norm=True, act="relu", name=f"{name}_up2_conv")

    # Output conv -> tanh
    out = tf.keras.layers.Conv2D(
        3, 7, strides=1, padding="same",
        kernel_initializer="glorot_uniform",
        name=f"{name}_out_conv",
    )(x)
    out = tf.keras.layers.Activation("tanh", name=f"{name}_tanh")(out)

    return tf.keras.Model(inp, out, name=name)

# -------------------------
# PatchGAN Discriminator (logits)
# -------------------------
def build_discriminator(name="discriminator", img_size=256, base_filters=64):
    """
    PatchGAN discriminator.
    Input:  (img_size, img_size, 3) in [-1, 1]
    Output: (H', W', 1) logits map (no sigmoid) for BCEWithLogits
    """
    inp = tf.keras.layers.Input(shape=(img_size, img_size, 3), name=f"{name}_input")

    # First layer usually without norm
    x = conv_norm_act(inp, base_filters, kernel_size=4, strides=2, padding="same",
                      use_norm=False, act="lrelu", name=f"{name}_c1")
    x = conv_norm_act(x, base_filters * 2, kernel_size=4, strides=2, padding="same",
                      use_norm=True, act="lrelu", name=f"{name}_c2")

    x = conv_norm_act(x, base_filters * 4, kernel_size=4, strides=2, padding="same",
                      use_norm=True, act="lrelu", name=f"{name}_c3")
    x = conv_norm_act(x, base_filters * 8, kernel_size=4, strides=1, padding="same",
                      use_norm=True, act="lrelu", name=f"{name}_c4")

    # Final conv to 1 channel logits map (no activation)
    out = tf.keras.layers.Conv2D(
        1, 4, strides=1, padding="same",
        kernel_initializer="he_normal",
        name=f"{name}_out_logits",
    )(x)

    return tf.keras.Model(inp, out, name=name)

# -------------------------
# Instantiate CycleGAN components
# -------------------------
IMG_SIZE = 256
BASE_FILTERS_G = 16
BASE_FILTERS_D = 16
N_RES_BLOCKS = 4  # from 3 to 6

G_photo2monet = build_generator("G_photo2monet", img_size=IMG_SIZE, base_filters=BASE_FILTERS_G, n_res_blocks=N_RES_BLOCKS)
F_monet2photo = build_generator("F_monet2photo", img_size=IMG_SIZE, base_filters=BASE_FILTERS_G, n_res_blocks=N_RES_BLOCKS)

D_monet = build_discriminator("D_monet", img_size=IMG_SIZE, base_filters=BASE_FILTERS_D)
D_photo = build_discriminator("D_photo", img_size=IMG_SIZE, base_filters=BASE_FILTERS_D)

# Print summaries (optional)
G_photo2monet.summary()
D_monet.summary()

# Quick shape check
x = tf.random.uniform([1, IMG_SIZE, IMG_SIZE, 3], minval=-1.0, maxval=1.0)
y = G_photo2monet(x)
d = D_monet(y)
print("Generator output:", y.shape, "range approx:", float(tf.reduce_min(y)), float(tf.reduce_max(y)))
print("Discriminator output (logits map):", d.shape)


## Loss Functions and Training Strategy

CycleGAN trains two generators and two discriminators, one pair per domain:

- **Generators**
  - \(G: X \rightarrow Y\) maps photos \(X\) to Monet-style images \(Y\)
  - \(F: Y \rightarrow X\) maps Monet-style images \(Y\) back to photos \(X\)

- **Discriminators**
  - \(D_Y\) distinguishes real Monet images from generated Monet-style images \(G(X)\)
  - \(D_X\) distinguishes real photos from generated photos \(F(Y)\)

### 1) Adversarial Loss (BCE with logits)

We use a binary cross-entropy loss with logits (no sigmoid in the discriminator output). Each discriminator produces a PatchGAN logits map, and the loss is computed over all patches and averaged.

For a discriminator \(D\):
- Real images should be classified as 1:
  \[
  \mathcal{L}_{D}^{real} = BCE(\mathbf{1}, D(real))
  \]
- Fake images should be classified as 0:
  \[
  \mathcal{L}_{D}^{fake} = BCE(\mathbf{0}, D(fake))
  \]
- Total discriminator loss:
  \[
  \mathcal{L}_{D} = \frac{1}{2}\left(\mathcal{L}_{D}^{real} + \mathcal{L}_{D}^{fake}\right)
  \]

For a generator (adversarial term), fake images should be classified as real:
\[
\mathcal{L}_{GAN}(G, D_Y) = BCE(\mathbf{1}, D_Y(G(X)))
\]
\[
\mathcal{L}_{GAN}(F, D_X) = BCE(\mathbf{1}, D_X(F(Y)))
\]

### 2) Cycle-Consistency Loss

Cycle-consistency encourages the translation to preserve the underlying content. After translating forward and back, we want to recover the original image:

\[
\mathcal{L}_{cyc}(G,F) =
\mathbb{E}_{x \sim X}\left[\|F(G(x)) - x\|_1\right] +
\mathbb{E}_{y \sim Y}\left[\|G(F(y)) - y\|_1\right]
\]

We use an \(L1\) reconstruction loss because it typically produces sharper reconstructions than \(L2\).

In [ ]:
# =========================
# CycleGAN - Training (CPU-only) + Load-or-Train + Persistent Save (Kaggle)
# RULES (as requested):
# - If saved generators exist -> LOAD them and SKIP training, SKIP saving.
# - Else -> (optionally restore checkpoint) TRAIN, then SAVE generators.
# =========================

import os
import time
from pathlib import Path
import tensorflow as tf

# -------------------------
# Persistent paths (Kaggle)
# -------------------------
SAVE_DIR = Path("/kaggle/working/saved_cyclegan_models")
G_DIR = SAVE_DIR / "G_photo2monet"
F_DIR = SAVE_DIR / "F_monet2photo"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# persistent checkpoints (optional resume)
CKPT_DIR = "/kaggle/working/checkpoints_cyclegan"
os.makedirs(CKPT_DIR, exist_ok=True)

# -------------------------
# Hyperparameters
# -------------------------
EPOCHS = 3
STEPS_PER_EPOCH = 25
BATCH_SIZE = 4
LR = 2e-4
BETA_1 = 0.5
LAMBDA_CYC = 10.0

# -------------------------
# Losses (BCE with logits + L1 cycle)
# -------------------------
bce_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)
mae = tf.keras.losses.MeanAbsoluteError()

def discriminator_loss(d_real, d_fake):
    real_loss = bce_logits(tf.ones_like(d_real), d_real)
    fake_loss = bce_logits(tf.zeros_like(d_fake), d_fake)
    return 0.5 * (real_loss + fake_loss)

def generator_gan_loss(d_fake):
    return bce_logits(tf.ones_like(d_fake), d_fake)

def cycle_loss(real, cycled):
    return mae(real, cycled)

# -------------------------
# Optimizers
# -------------------------
g_optimizer  = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=BETA_1)
f_optimizer  = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=BETA_1)
dx_optimizer = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=BETA_1)
dy_optimizer = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=BETA_1)

# -------------------------
# Metrics (epoch averages)
# -------------------------
m_d_photo = tf.keras.metrics.Mean(name="D_photo_loss")
m_d_monet = tf.keras.metrics.Mean(name="D_monet_loss")
m_g_adv   = tf.keras.metrics.Mean(name="G_adv_loss")
m_f_adv   = tf.keras.metrics.Mean(name="F_adv_loss")
m_cyc_x   = tf.keras.metrics.Mean(name="cycle_X_loss")
m_cyc_y   = tf.keras.metrics.Mean(name="cycle_Y_loss")
m_g_total = tf.keras.metrics.Mean(name="G_total_loss")

def reset_all_metrics():
    for m in [m_d_photo, m_d_monet, m_g_adv, m_f_adv, m_cyc_x, m_cyc_y, m_g_total]:
        m.reset_state()

# -------------------------
# Checkpointing (persistent)
# -------------------------
ckpt = tf.train.Checkpoint(
    G_photo2monet=G_photo2monet,
    F_monet2photo=F_monet2photo,
    D_photo=D_photo,
    D_monet=D_monet,
    g_optimizer=g_optimizer,
    f_optimizer=f_optimizer,
    dx_optimizer=dx_optimizer,
    dy_optimizer=dy_optimizer,
)
ckpt_manager = tf.train.CheckpointManager(ckpt, CKPT_DIR, max_to_keep=3)

# -------------------------
# SavedModel helpers (generators)
# -------------------------
def saved_generators_exist():
    # A SavedModel directory contains "saved_model.pb" (and variables/)
    return (G_DIR / "saved_model.pb").exists() and (F_DIR / "saved_model.pb").exists()

def load_saved_generators():
    print("✅ Found saved generators. Loading and skipping training...")
    G_loaded = tf.keras.models.load_model(G_DIR, compile=False)
    F_loaded = tf.keras.models.load_model(F_DIR, compile=False)
    return G_loaded, F_loaded

def save_generators(G, F):
    print("💾 Saving generators to:", SAVE_DIR)
    tf.keras.models.save_model(G, G_DIR, overwrite=True, include_optimizer=False)
    tf.keras.models.save_model(F, F_DIR, overwrite=True, include_optimizer=False)
    print("✅ Saved generators:", G_DIR, "and", F_DIR)

# -------------------------
# One training step
# -------------------------
@tf.function
def train_step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        fake_y = G_photo2monet(real_x, training=True)    # X -> Y
        fake_x = F_monet2photo(real_y, training=True)    # Y -> X

        cycled_x = F_monet2photo(fake_y, training=True)  # X -> Y -> X
        cycled_y = G_photo2monet(fake_x, training=True)  # Y -> X -> Y

        d_real_x = D_photo(real_x, training=True)
        d_fake_x = D_photo(fake_x, training=True)

        d_real_y = D_monet(real_y, training=True)
        d_fake_y = D_monet(fake_y, training=True)

        dx_loss = discriminator_loss(d_real_x, d_fake_x)
        dy_loss = discriminator_loss(d_real_y, d_fake_y)

        g_adv = generator_gan_loss(d_fake_y)
        f_adv = generator_gan_loss(d_fake_x)

        cyc_x = cycle_loss(real_x, cycled_x)
        cyc_y = cycle_loss(real_y, cycled_y)
        g_total = g_adv + f_adv + (LAMBDA_CYC * (cyc_x + cyc_y))

    g_grads  = tape.gradient(g_total, G_photo2monet.trainable_variables)
    f_grads  = tape.gradient(g_total, F_monet2photo.trainable_variables)
    dx_grads = tape.gradient(dx_loss,  D_photo.trainable_variables)
    dy_grads = tape.gradient(dy_loss,  D_monet.trainable_variables)

    g_optimizer.apply_gradients(zip(g_grads,  G_photo2monet.trainable_variables))
    f_optimizer.apply_gradients(zip(f_grads,  F_monet2photo.trainable_variables))
    dx_optimizer.apply_gradients(zip(dx_grads, D_photo.trainable_variables))
    dy_optimizer.apply_gradients(zip(dy_grads, D_monet.trainable_variables))

    m_d_photo.update_state(dx_loss)
    m_d_monet.update_state(dy_loss)
    m_g_adv.update_state(g_adv)
    m_f_adv.update_state(f_adv)
    m_cyc_x.update_state(cyc_x)
    m_cyc_y.update_state(cyc_y)
    m_g_total.update_state(g_total)

# -------------------------
# Training loop
# -------------------------
def train(epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH):
    photo_iter = iter(photo_ds)
    monet_iter = iter(monet_ds)

    for epoch in range(1, epochs + 1):
        reset_all_metrics()
        t0 = time.time()

        for step in range(1, steps_per_epoch + 1):
            real_x = next(photo_iter)
            real_y = next(monet_iter)

            train_step(real_x, real_y)

            if step % 10 == 0 or step == 1:
                print(
                    f"Epoch {epoch}/{epochs} | Step {step}/{steps_per_epoch} | "
                    f"Dx={m_d_photo.result():.4f} Dy={m_d_monet.result():.4f} | "
                    f"G_adv={m_g_adv.result():.4f} F_adv={m_f_adv.result():.4f} | "
                    f"cycX={m_cyc_x.result():.4f} cycY={m_cyc_y.result():.4f} | "
                    f"G_total={m_g_total.result():.4f}"
                )

        ckpt_path = ckpt_manager.save(checkpoint_number=epoch)
        dt = time.time() - t0
        print(
            f"\n✅ Epoch {epoch} done in {dt/60:.2f} min | saved: {ckpt_path}\n"
            f"Final epoch metrics: "
            f"Dx={m_d_photo.result():.4f}, Dy={m_d_monet.result():.4f}, "
            f"G_adv={m_g_adv.result():.4f}, F_adv={m_f_adv.result():.4f}, "
            f"cycX={m_cyc_x.result():.4f}, cycY={m_cyc_y.result():.4f}, "
            f"G_total={m_g_total.result():.4f}\n"
        )

# =========================
# MAIN: load-or-train
# =========================
if saved_generators_exist():
    G_photo2monet, F_monet2photo = load_saved_generators()
else:
    # optional resume from checkpoint
    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint).expect_partial()
        print("✅ Restored from checkpoint:", ckpt_manager.latest_checkpoint)
    else:
        print("Initializing from scratch (no saved generators, no checkpoint).")

    train()
    # Save ONLY because they did not exist
    # save_generators(G_photo2monet, F_monet2photo)


In [ ]:
# =========================
# Visualization: Photo -> Monet (G) -> Photo (F)
# Row 1: original photos (X)
# Row 2: G_photo2monet(X)
# Row 3: F_monet2photo(G_photo2monet(X))  (cycle back)
# =========================

import matplotlib.pyplot as plt
import tensorflow as tf

def denorm_m11_to_01(img):
    # img in [-1, 1] -> [0, 1]
    return tf.clip_by_value((img + 1.0) / 2.0, 0.0, 1.0)

# Take one batch from photo_ds
real_x = next(iter(photo_ds))  # shape: (B, 256, 256, 3), in [-1, 1]
real_x_4 = real_x[:4]

# Forward + cycle 
fake_y_4 = G_photo2monet(real_x_4, training=False)          # X -> Y
cycled_x_4 = F_monet2photo(fake_y_4, training=False)        # X -> Y -> X

# Convert to displayable [0,1]
row1 = denorm_m11_to_01(real_x_4)
row2 = denorm_m11_to_01(fake_y_4)
row3 = denorm_m11_to_01(cycled_x_4)

# Plot 3x4
plt.figure(figsize=(16, 10))

for i in range(4):
    # Row 1: originals
    ax = plt.subplot(3, 4, i + 1)
    ax.imshow(row1[i].numpy())
    ax.set_title("Original (Photo)")
    ax.axis("off")

    # Row 2: G output
    ax = plt.subplot(3, 4, 4 + i + 1)
    ax.imshow(row2[i].numpy())
    ax.set_title("G: Photo → Monet")
    ax.axis("off")

    # Row 3: F output on G output (cycled back)
    ax = plt.subplot(3, 4, 8 + i + 1)
    ax.imshow(row3[i].numpy())
    ax.set_title("F: Monet → Photo (Cycle)")
    ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# =========================
# Generate final Monet-style images with trained generator (CPU-only)
# Produces N images as JPG (256x256) from PHOTO domain using G_photo2monet
# =========================

OUT_DIR = Path("./generated_monet_jpg")
OUT_DIR.mkdir(parents=True, exist_ok=True)

N_GEN = 7000

# Ensure we use the trained generator in inference mode
G_photo2monet.trainable = False

def m11_to_uint8(img_m11):
    """
    img_m11: float tensor in [-1,1], shape (H,W,3)
    returns uint8 array in [0,255]
    """
    img01 = tf.clip_by_value((img_m11 + 1.0) / 2.0, 0.0, 1.0)
    img255 = tf.cast(tf.round(img01 * 255.0), tf.uint8)
    return img255.numpy()

# Iterate over photos and generate
photo_iter = iter(photo_ds)

count = 0
while count < N_GEN:
    batch = next(photo_iter)  # (B,256,256,3) in [-1,1]
    fake_y = G_photo2monet(batch, training=False)  # (B,256,256,3) in [-1,1]
    if count % 10  == 0:
        print("count: ", count)

    b = fake_y.shape[0]
    for i in range(b):
        if count >= N_GEN:
            break
        img = m11_to_uint8(fake_y[i])
        Image.fromarray(img).save(OUT_DIR / f"monet_{count:05d}.jpg", quality=95)
        count += 1

print(f"Saved {count} generated images to: {OUT_DIR.resolve()}")


In [ ]:
import zipfile
from pathlib import Path

zip_path = "images.zip"
img_dir = Path("./generated_monet_jpg")

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for img_path in img_dir.glob("*.jpg"):
        zipf.write(img_path, arcname=img_path.name)

print(f"Created submission file: {zip_path}")


## Results

The trained CycleGAN is able to generate images that partially capture the visual style of Claude Monet.  
The generated images show noticeable changes in color distribution and texture compared to the original photographs, with softer edges and painterly color blending.

Due to limited training time and computational constraints, the generated images do not fully match the complexity and richness of real Monet paintings. However, the outputs demonstrate that the model has learned meaningful stylistic features from the training data.


## Discussion

This project demonstrates the application of Generative Adversarial Networks to artistic style generation using an unpaired image dataset.  
CycleGAN was selected because it enables image-to-image translation without paired examples, which aligns with the structure of the dataset.

The main limitations of this work include a small number of training epochs, reduced model capacity, and CPU-only training. These constraints were chosen intentionally to keep the project lightweight and focused on conceptual understanding rather than leaderboard optimization.


## Conclusion

In this mini-project, a CycleGAN model was implemented and trained to generate Monet-style images from photographs.  
The project provided hands-on experience with GAN architectures, adversarial training, and unsupervised image translation.

While the generated images are not of professional artistic quality, the results confirm that the model successfully learned stylistic patterns from the dataset. This work highlights both the creative potential of GANs and the importance of sufficient training and computational resources for high-quality image generation.
